# Test Car Recognition

In [0]:
import os
from scipy.io import loadmat
from pathlib import Path
import pandas as pd
import cv2 as cv
from tqdm import tqdm

import fastai
from fastai.vision import *

from sklearn.metrics import classification_report

ROOT_DIR = Path('/content')
DATA_DIR = ROOT_DIR/'data'
META_DIR= ROOT_DIR/'devkit'

TRAINED_MODEL = 'https://github.com/nithiroj/car-recognition/releases/download/v1.0/stage-2-152-c.pkl'

## Download Testing Data and Trained Model

In [0]:
# Load once

!wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
!wget http://imagenet.stanford.edu/internal/car196/cars_test_annos_withlabels.mat
!wget {TRAINED_MODEL}

--2019-06-17 12:34:20--  https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330960 (323K) [application/x-gzip]
Saving to: ‘car_devkit.tgz’

car_devkit.tgz      100%[===================>] 323.20K  1.11MB/s    in 0.3s    

2019-06-17 12:34:20 (1.11 MB/s) - ‘car_devkit.tgz’ saved [330960/330960]

--2019-06-17 12:34:22--  http://imagenet.stanford.edu/internal/car196/cars_test.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977350468 (932M) [application/x-gzip]
Saving to: ‘cars_test.tgz’

cars_test.tgz       100%[===================>] 932.07M  22.9MB/s    in 41s     

2019-06-17 12:35:02 (23.0 MB/s) - ‘cars_test.tgz’ saved [97

In [0]:
!tar -zxf car_devkit.tgz
!tar -zxf cars_test.tgz 

In [0]:
def load_annos(meta_dir, mat_file, labels=True):
  '''Load annotation data'''
  
  annos = {}
  keys = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'fname']
  
  if labels:
    keys = keys + ['class']

  data = loadmat(meta_dir/mat_file)
  data = data['annotations']

  for key in keys:
    values = data[key][0]
    if key == 'fname':
      values = [y for x in values for y in x]
    else:
      values = [y[0] for x in values for y in x]
    
    annos[key] = values
    
  return annos

In [0]:
def load_classes(meta_dir):
  '''List of classes
  :param meta_dir: 
  :return: idx to class dict
  '''
  classes = loadmat(meta_dir/'cars_meta.mat')
  classes = classes['class_names'][0]
  classes = [y for x in classes for y in x]
  idx_to_class = {idx+1:clss for idx, clss in enumerate(classes)}
  
  return idx_to_class

In [5]:
test_annos = load_annos(META_DIR, mat_file='cars_test_annos.mat', labels=False)
test_annos_withlabels = load_annos(ROOT_DIR, mat_file='cars_test_annos_withlabels.mat')
classes = load_classes(META_DIR)
classes[71], classes[119]

('Chevrolet Express Van 2007', 'GMC Savana Van 2012')

## Preprocess Images

In [0]:
def crop_images(src_folder, annos):
  
  margin = 16
  num_samples = len(annos['fname'])
  
  for i in tqdm(range(num_samples)):
    fname = annos['fname'][i]
    x1 = annos['bbox_x1'][i]
    y1 = annos['bbox_y1'][i] 
    x2 = annos['bbox_x2'][i] 
    y2 = annos['bbox_y2'][i] 
    
    src_path = ROOT_DIR/src_folder/fname
    src_img = cv.imread(str(src_path))
    h, w = src_img.shape[:2]
    
    x1 = max(0, x1 - margin)
    y1 = max(0, y1 - margin)
    x2 = min(x2 + margin, w)
    y2 = min(y2 + margin, h)
    
    dst_folder = DATA_DIR/f'crop_{src_folder}'
    dst_path = dst_folder/fname
    os.makedirs(dst_folder, exist_ok=True)
    
    crop_img = src_img[y1:y2, x1:x2]
    cv.imwrite(str(dst_path), crop_img)

In [7]:
crop_images('cars_test',test_annos)

100%|██████████| 8041/8041 [01:15<00:00, 106.05it/s]


## Predictions

In [0]:
def batch_predict(img_folder, model_dir, model_file):
  preds = {}
  learn = load_learner(model_dir, file=model_file)
  
  for img_file in tqdm(os.listdir(img_folder)):
    img_path = os.path.join(img_folder, img_file)
    img = open_image(img_path)
    pred_class, _, prob = learn.predict(img)
    preds[img_file] = (int(str(pred_class)), prob.max().item())
  
  return preds

### Load  Model

In [0]:
img_folder = DATA_DIR/'crop_cars_test'
model_dir = ROOT_DIR
model_file = 'stage-2-152-c.pkl'

In [10]:
preds = batch_predict(img_folder, model_dir, model_file)

100%|██████████| 8041/8041 [06:20<00:00, 21.13it/s]


In [0]:
def predictions(preds, classes):
  print('Image, Confidence, Label, Car_Model')
  for fname in sorted(preds):
    prob = preds[fname][1]
    label = preds[fname][0]
    model = classes[label]
    print('{}, {:4f}, {}, {}'.format(fname, prob, label, model))
    

In [13]:
predictions(preds, classes)

Image, Confidence, Label, Car_Model
00001.jpg, 0.989304, 181, Suzuki Aerio Sedan 2007
00002.jpg, 0.994412, 103, Ferrari 458 Italia Convertible 2012
00003.jpg, 0.981209, 145, Jeep Patriot SUV 2012
00004.jpg, 0.953416, 187, Toyota Camry Sedan 2012
00005.jpg, 0.964303, 185, Tesla Model S Sedan 2012
00006.jpg, 0.965354, 78, Chrysler Town and Country Minivan 2012
00007.jpg, 0.973375, 118, GMC Terrain SUV 2012
00008.jpg, 0.980882, 165, Mercedes-Benz S-Class Sedan 2012
00009.jpg, 0.939814, 32, BMW X5 SUV 2007
00010.jpg, 0.998578, 60, Chevrolet HHR SS 2010
00011.jpg, 0.985266, 49, Buick Verano Sedan 2012
00012.jpg, 0.998526, 108, Ford Freestar Minivan 2007
00013.jpg, 0.922031, 116, Ford E-Series Wagon Van 2012
00014.jpg, 0.850739, 135, Hyundai Elantra Sedan 2007
00015.jpg, 0.971915, 83, Dodge Caliber Wagon 2012
00016.jpg, 0.999334, 51, Cadillac CTS-V Sedan 2012
00017.jpg, 0.996239, 154, Land Rover Range Rover SUV 2012
00018.jpg, 0.814168, 33, BMW X6 SUV 2012
00019.jpg, 0.456186, 19, Audi TT Ha

In [0]:
def report(preds, test_annos_withlabels):
  test_labels = test_annos_withlabels['class']
  fnames = test_annos_withlabels['fname']
  test_preds = [preds[fname][0] for fname in fnames]
  print(classification_report(test_labels, test_preds, digits=4))


In [15]:
report(preds, test_annos_withlabels)

              precision    recall  f1-score   support

           1     0.9333    0.9545    0.9438        44
           2     0.7778    0.8750    0.8235        32
           3     0.8077    0.9767    0.8842        43
           4     1.0000    0.9524    0.9756        42
           5     0.9706    0.8250    0.8919        40
           6     0.9565    1.0000    0.9778        44
           7     0.9714    0.8718    0.9189        39
           8     0.7750    0.6889    0.7294        45
           9     0.7333    0.8049    0.7674        41
          10     0.9375    0.9091    0.9231        33
          11     0.8537    0.9211    0.8861        38
          12     1.0000    0.8889    0.9412        36
          13     0.7170    0.9268    0.8085        41
          14     0.8182    0.6429    0.7200        42
          15     0.8500    0.7907    0.8193        43
          16     0.7500    0.7674    0.7586        43
          17     0.6667    0.6500    0.6582        40
          18     0.6429    